In [1]:
# 패키지 import
import os.path as osp
from PIL import Image

import torch.utils.data as data

# 데이터셋과 데이터 로더 구현
## image 데이터 및 어노테이션 데이터 파일의 경로 리스트 작성

In [13]:
def make_datapath_list(rootpath):
    imagepath_template = osp.join(rootpath, "JPEGImages", "%s.jpg")
    annopath_template = osp.join(rootpath,"SegmentationClass","%s.png")

    train_id_names = osp.join(rootpath+"ImageSets/Segmentation/train.txt")
    val_id_names = osp.join(rootpath+"ImageSets/Segmentation/val.txt")

    train_img_list = list()
    train_anno_list = list()

    for line in open(train_id_names):
        file_id = line.strip()
        img_path = (imagepath_template % file_id)
        anno_path = (annopath_template % file_id)
        train_img_list.append(img_path)
        train_anno_list.append(anno_path)

    val_img_list = list()
    val_anno_list = list()

    for line in open(val_id_names):
        file_id = line.strip()
        img_path = (imagepath_template % file_id)
        anno_path = (annopath_template % file_id)
        val_img_list.append(img_path)
        val_anno_list.append(anno_path)

    return train_img_list, train_anno_list, val_img_list, val_anno_list

rootpath = "./data/VOCdevkit/VOC2012/"

train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath=rootpath)
print(train_img_list[0])
print(train_anno_list[0])

./data/VOCdevkit/VOC2012/JPEGImages/2007_000032.jpg
./data/VOCdevkit/VOC2012/SegmentationClass/2007_000032.png


## 데이터셋 작성

In [14]:
from utils.data_augumentation import *

class DataTransform():
    def __init__(self, input_size, color_mean, color_std):
        self.data_transform={
            "train":Compose([
                Scale(scale=[0.5,1.5]),
                RandomRotation(angle=[-10,10]),
                RandomMirror(),
                Resize(input_size),
                Normalize_Tensor(color_mean, color_std)
            ]),
            "val":Compose([
                Resize(input_size),
                Normalize_Tensor(color_mean, color_std)
            ])
        }

    def __call__(self, phase, img, anno_class_img):
        return self.data_transform[phase](img, anno_class_img)

In [16]:
class VOCDataset(data.Dataset):
    def __init__(self, img_list, anno_list, phase, transform):
        self.img_list = img_list
        self.anno_list = anno_list
        self.phase = phase
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, index):
        img, anno_class_img = self.pull_item(index)
        return img, anno_class_img

    def pull_item(self, index):
        image_file_path = self.img_list[index]
        img = Image.open(image_file_path)

        anno_file_path = self.anno_list[index]
        anno_class_img = Image.open(anno_file_path)

        img, anno_class_img = self.transform(self.phase, img, anno_class_img)

        return img, anno_class_img


color_mean = (0.485, 0.456, 0.406)
color_std = (0.229, 0.224, 0.225)

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(input_size=475, color_mean=color_mean, color_std=color_std))
val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(input_size=475, color_mean=color_mean, color_std=color_std))

print(val_dataset.__getitem__(0)[0].shape)

torch.Size([3, 475, 475])


## 데이터 로더 작성

In [17]:
batch_size = 8

train_dataloader = data.DataLoader(train_dataset, batch_size, shuffle=True)
val_dataloader = data.DataLoader(val_dataset, batch_size, shuffle=False)

dataloaders_dict = {"train":train_dataloader, "val":val_dataloader}

batch_iterator = iter(dataloaders_dict["val"])
images, anno_class_images = next(batch_iterator)
print(images.size())

torch.Size([8, 3, 475, 475])
